# Assess data quality

In [ ]:
#imports
import pylidc as pl
from pylidc.utils import consensus
import SimpleITK as sitk
from radiomics import featureextractor
import pandas as pd

In [1]:
#lista com ids dos pacientes que têm diagnóstico
data = pd.read_excel("tcia-diagnosis-data-2012-04-20.xls")
column_names = data.columns.tolist()
#column_names
ids_list = data["TCIA Patient ID"].tolist()

157

In [2]:
ids_list

['LIDC-IDRI-0068',
 'LIDC-IDRI-0071',
 'LIDC-IDRI-0072',
 'LIDC-IDRI-0088',
 'LIDC-IDRI-0090',
 'LIDC-IDRI-0091',
 'LIDC-IDRI-0100',
 'LIDC-IDRI-0118',
 'LIDC-IDRI-0124',
 'LIDC-IDRI-0129',
 'LIDC-IDRI-0135',
 'LIDC-IDRI-0137',
 'LIDC-IDRI-0138',
 'LIDC-IDRI-0143',
 'LIDC-IDRI-0149',
 'LIDC-IDRI-0159',
 'LIDC-IDRI-0161',
 'LIDC-IDRI-0162',
 'LIDC-IDRI-0163',
 'LIDC-IDRI-0164',
 'LIDC-IDRI-0165',
 'LIDC-IDRI-0166',
 'LIDC-IDRI-0167',
 'LIDC-IDRI-0168',
 'LIDC-IDRI-0169',
 'LIDC-IDRI-0171',
 'LIDC-IDRI-0173',
 'LIDC-IDRI-0174',
 'LIDC-IDRI-0175',
 'LIDC-IDRI-0176',
 'LIDC-IDRI-0178',
 'LIDC-IDRI-0179',
 'LIDC-IDRI-0180',
 'LIDC-IDRI-0181',
 'LIDC-IDRI-0182',
 'LIDC-IDRI-0183',
 'LIDC-IDRI-0184',
 'LIDC-IDRI-0185',
 'LIDC-IDRI-0186',
 'LIDC-IDRI-0187',
 'LIDC-IDRI-0188',
 'LIDC-IDRI-0189',
 'LIDC-IDRI-0190',
 'LIDC-IDRI-0191',
 'LIDC-IDRI-0192',
 'LIDC-IDRI-0193',
 'LIDC-IDRI-0194',
 'LIDC-IDRI-0197',
 'LIDC-IDRI-0198',
 'LIDC-IDRI-0200',
 'LIDC-IDRI-0202',
 'LIDC-IDRI-0203',
 'LIDC-IDRI-

In [7]:
#extrair as features do dataset

# Configurar o extrator de características do PyRadiomics sem o arquivo params.yaml
extractor = featureextractor.RadiomicsFeatureExtractor()

# Consultar todas as varreduras com anotações
scans_with_annotations = pl.query(pl.Scan).filter(pl.Scan.annotations.any()).all()

# Lista para armazenar as características
features_list = []

# Lista para armazenar o ID do paciente correspondente a cada nódulo
patient_ids = []

# Variável para criar IDs únicos para os nódulos
nodule_id_counter = 1

# Iterar através de todas as varreduras com anotações
for scan in scans_with_annotations:
    # Obter o ID do paciente
    patient_id = scan.patient_id

    # Check if the patient ID is in the desired list
    if patient_id in ids_list:
        # Clusterize as anotações para a varredura e pegue todas
        nods = scan.cluster_annotations()

        # Iterar através de todos os nódulos do paciente
        for anns in nods:
            # Check if the current nodule has annotations
            if anns:
                # Realize a consolidação de consenso e nível de acordo de 50%
                cmask, cbbox, masks = consensus(anns, clevel=0.5, pad=[(20, 20), (20, 20), (0, 0)])

                # Converter a matriz de pixels para uma imagem SimpleITK
                image = sitk.GetImageFromArray(cmask.astype(float))

                # Extrair características radiômicas usando PyRadiomics
                features = extractor.execute(image, image, label=1)  # Usar rótulo 1 para o nódulo

                # Adicionar o ID do paciente à lista
                patient_ids.append(patient_id)

                # Adicionar um ID único para o nódulo
                features['Nodule_ID'] = f'Nodule_{nodule_id_counter}'
                nodule_id_counter += 1

                # Adicionar as características à lista
                features_list.append(features)

# Converter a lista de características em um DataFrame do pandas
features_df = pd.DataFrame(features_list)

# Adicionar uma coluna "Patient_ID" ao DataFrame
features_df['Patient_ID'] = patient_ids

# Selecionar apenas as características desejadas
selected_features = features_df[['Patient_ID',
                                 'Nodule_ID', 
                                 'original_shape_Elongation', 
                                 'original_shape_Sphericity', 
                                 'original_shape_VoxelVolume',
                                 'original_shape_SurfaceArea', 
                                 'original_shape_Maximum2DDiameterSlice', 
                                 'original_firstorder_Entropy', 
                                 'original_firstorder_Energy', 
                                 'original_glrlm_LongRunEmphasis', 
                                 'original_glrlm_ShortRunEmphasis']]
# Salvar as características selecionadas em um arquivo CSV
selected_features.to_csv('selected_radiomics_features3.csv', index=False)


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

Failed to reduce all groups to <= 4 Annotations.
Some nodules may be close and must be grouped manually.


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

Failed to reduce all groups to <= 4 Annotations.
Some nodules may be close and must be grouped manually.


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

Failed to reduce all groups to <= 4 Annotations.
Some nodules may be close and must be grouped manually.


GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

PermissionError: [Errno 13] Permission denied: 'selected_radiomics_features3.csv'

In [9]:
#lista de pacientes que têm diagnóstico de dois nódulos

# Load your DataFrame from your CSV file
df = pd.read_excel('tcia-diagnosis-data-2012-04-20.xls')

# Replace 'Column1' and 'Column2' with the actual column names in your DataFrame
column1 = 'TCIA Patient ID'
column2 = 'Nodule 2\nDiagnosis at the Nodule Level \n0=Unknown\n1=benign or non-malignant disease\n2= malignant, primary lung cancer\n3 = malignant metastatic)\n'

# Filter rows where values in Column2 are not null and get values from Column1
filtered_values = df.loc[df[column2].notnull(), column1]

# Print the filtered values
filtered_values=filtered_values.tolist()
filtered_values


20

In [16]:
#csv com os dados de dois nódulos por paciente

# Load your DataFrame from your CSV file
df = pd.read_csv('selected_radiomics_features3.csv')

# Replace 'Column1' with the actual column name containing repeated values
column_name = 'Patient_ID'

# Keep only the first two rows for each unique value in the specified column
filtered_df = df.groupby(column_name).head(2)

# Save the filtered DataFrame to a new CSV file
filtered_df.to_csv('selected_radiomics_features.csv', index=False)
df = pd.read_csv('selected_radiomics_features.csv')
print(df)


         Patient_ID   Nodule_ID  original_shape_Elongation   
0    LIDC-IDRI-0068    Nodule_1                   0.802433  \
1    LIDC-IDRI-0068    Nodule_2                   0.837491   
2    LIDC-IDRI-0072    Nodule_7                   0.693080   
3    LIDC-IDRI-0088   Nodule_10                   0.861460   
4    LIDC-IDRI-0088    Nodule_8                   0.854878   
..              ...         ...                        ...   
225  LIDC-IDRI-1004  Nodule_356                   0.941111   
226  LIDC-IDRI-1004  Nodule_357                   0.734604   
227  LIDC-IDRI-1010  Nodule_355                   0.597327   
228  LIDC-IDRI-1011  Nodule_351                   0.721525   
229  LIDC-IDRI-1011  Nodule_352                   0.655030   

     original_shape_Sphericity  original_shape_VoxelVolume   
0                     0.829843                       121.0  \
1                     0.798718                       212.0   
2                     0.471066                     12014.0   
3      

In [14]:
#csv com os dados de apenas do segundo nódulo dos pacientes com diagnóstico

# Load your DataFrame from your CSV file
df = pd.read_csv('selected_radiomics_features.csv')

# Replace 'Patient_ID' with the actual column name containing the IDs
column_name = 'Patient_ID'

# Create a list of 'Patient_ID' values that should be excluded from filtering
#filtered_values = ['ID1', 'ID2', 'ID3']  # Replace with your filtered IDs

# Filter rows to keep only one value for each 'Patient_ID' not in the filtered list
filtered_df = df[df[column_name].isin(filtered_values)]
filtered_df = filtered_df.groupby(column_name, as_index=False).last()

# Save the filtered DataFrame to a new CSV file
filtered_df.to_csv('filtered_csv_file.csv', index=False)

# Reload and print the filtered DataFrame
df = pd.read_csv('filtered_csv_file.csv')
print(df)




        Patient_ID   Nodule_ID  original_shape_Elongation   
0   LIDC-IDRI-0171  Nodule_102                   0.812866  \
1   LIDC-IDRI-0185  Nodule_140                   0.761163   
2   LIDC-IDRI-0190  Nodule_165                   0.468780   
3   LIDC-IDRI-0194  Nodule_174                   0.874753   
4   LIDC-IDRI-0203  Nodule_181                   0.563345   
5   LIDC-IDRI-0210  Nodule_184                   0.838137   
6   LIDC-IDRI-0223  Nodule_198                   0.881455   
7   LIDC-IDRI-0234  Nodule_208                   0.853028   
8   LIDC-IDRI-0258  Nodule_243                   0.825334   
9   LIDC-IDRI-0265  Nodule_250                   0.616792   
10  LIDC-IDRI-0267  Nodule_253                   0.889258   
11  LIDC-IDRI-0272  Nodule_257                   0.710713   
12  LIDC-IDRI-0285  Nodule_273                   0.603448   
13  LIDC-IDRI-0289  Nodule_280                   0.661799   
14  LIDC-IDRI-0290  Nodule_283                   0.761269   
15  LIDC-IDRI-0332  Nodu

In [15]:
#csv com os dados de apenas do primeiro nódulo dos pacientes com diagnóstico

# Load your DataFrame from your CSV file
df = pd.read_csv('selected_radiomics_features.csv')

# Replace 'Column1' with the actual column name containing repeated values
column_name = 'Patient_ID'

# Keep only the first two rows for each unique value in the specified column
filtered_df = df.groupby(column_name).head(1)

# Save the filtered DataFrame to a new CSV file
filtered_df.to_csv('selected_radiomics_features.csv', index=False)
df = pd.read_csv('selected_radiomics_features.csv')
print(df)

         Patient_ID   Nodule_ID  original_shape_Elongation   
0    LIDC-IDRI-0068    Nodule_1                   0.802433  \
1    LIDC-IDRI-0072    Nodule_7                   0.693080   
2    LIDC-IDRI-0088   Nodule_10                   0.861460   
3    LIDC-IDRI-0090   Nodule_11                   0.727102   
4    LIDC-IDRI-0091   Nodule_12                   0.749062   
..              ...         ...                        ...   
137  LIDC-IDRI-0994  Nodule_364                   0.796769   
138  LIDC-IDRI-1002  Nodule_360                   0.930841   
139  LIDC-IDRI-1004  Nodule_356                   0.941111   
140  LIDC-IDRI-1010  Nodule_355                   0.597327   
141  LIDC-IDRI-1011  Nodule_351                   0.721525   

     original_shape_Sphericity  original_shape_VoxelVolume   
0                     0.829843                       121.0  \
1                     0.471066                     12014.0   
2                     0.692884                        89.0   
3      

In [17]:
#csv com os dados dos dois anteriores

# Load the first CSV file into a DataFrame
df1 = pd.read_csv('selected_radiomics_features.csv')

# Load the second CSV file into another DataFrame
df2 = pd.read_csv('filtered_csv_file.csv')

# Merge the two DataFrames
merged_df = pd.concat([df1, df2], ignore_index=True)
merged_df = merged_df.sort_values(by='Patient_ID')
# Save the merged DataFrame to a new CSV file
merged_df.to_csv('merged_file.csv', index=False)

# Optionally, you can also print or inspect the merged DataFrame
print(merged_df)


         Patient_ID   Nodule_ID  original_shape_Elongation   
0    LIDC-IDRI-0068    Nodule_1                   0.802433  \
1    LIDC-IDRI-0072    Nodule_7                   0.693080   
2    LIDC-IDRI-0088   Nodule_10                   0.861460   
3    LIDC-IDRI-0090   Nodule_11                   0.727102   
4    LIDC-IDRI-0091   Nodule_12                   0.749062   
..              ...         ...                        ...   
138  LIDC-IDRI-1002  Nodule_360                   0.930841   
139  LIDC-IDRI-1004  Nodule_356                   0.941111   
140  LIDC-IDRI-1010  Nodule_355                   0.597327   
141  LIDC-IDRI-1011  Nodule_351                   0.721525   
159  LIDC-IDRI-1011  Nodule_352                   0.655030   

     original_shape_Sphericity  original_shape_VoxelVolume   
0                     0.829843                       121.0  \
1                     0.471066                     12014.0   
2                     0.692884                        89.0   
3      